### Hyperparameter tuning with grid search

In [1]:
import torch
from core.util.save_model import save_model
from sklearn.model_selection import ParameterGrid
from core.models import LSTM
from core.models.model_training import blocked_training
from core.util.hyperparameter_configuration import get_hyperparameter_configuration
from core.util.early_stop import EarlyStop

Set ML model, loss function and hyperparameters that that will be tested.

In [2]:
gridsearch_params = {
    "learning_rate": [0.001, 0.003, 0.005],
    "batch_size": [32, 64, 128],
    "lookback": [36, 48, 96],
    "num_layers": [1, 2],
}

# Extract hyperparameters configuration that will not be tuned upon
hidden_size, EPOCHS, horizon, lookback, loss_function, dropout_rate, folds = (
    get_hyperparameter_configuration()
)

# Specific to experiment
features = {"Consumption": True}
experiment_name = "0_LSTM_trefor_park"
model_used = LSTM

Use CUDA (GPU) if available.

In [3]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cpu'

Train a model with specified hyperparameters

In [4]:
def train_with_params(params: dict) -> tuple[float, LSTM]:
    """Train model with the specified hyperparameters."""
    # Extract hyperparameters
    batch_size = params["batch_size"]
    learning_rate = params["learning_rate"]
    num_layers = params["num_layers"]
    lookback = params["lookback"]

    # Initialize model
    model = model_used(
        input_size=len(features),
        hidden_size=hidden_size,
        num_layers=num_layers,
        dropout_rate=dropout_rate,
        horizon=horizon,
        lookback=lookback,
    ).to(device)

    _, val_loss, best_model = blocked_training(
        model=model,
        learning_rate=learning_rate,
        device=device,
        batch_size=batch_size,
        early_stopper=EarlyStop(5, 0.05),
    )

    return min(val_loss), best_model

Iterate over all hyperparameters and train a model for each combination.

In [5]:
best_loss = float("inf")
best_params = None
best_model = None

for params in ParameterGrid(gridsearch_params):
    v_loss, model = train_with_params(params)
    print(params, v_loss)
    if v_loss < best_loss:
        best_loss = v_loss
        best_params = params
        best_model = model

save_model(best_model, experiment_name)
print("Best Hyperparameters:", best_params)
print("Best Validation Loss:", best_loss)

Iterating epochs:   0%|          | 0/10 [00:00<?, ?it/s]

KeyboardInterrupt: 